### Libraries
Besides [Numpy](https://numpy.org/) and the Pyplot interface to [Matplotlib](https://matplotlib.org/), we will need [OpenCV](https://opencv.org/).

In [45]:
from matplotlib import pyplot as plt

import os

import numpy as np

import cv2 # OpenCV
print (cv2.__version__)

4.10.0


In [46]:
folderpath = './images'

1. Calcolare omografia per ogni immagine e c'è già la funzione di Edi

In [47]:
grid_size = (8,11)
def calculate_homography(image)->np.ndarray:
    return_value, corners = cv2.findChessboardCorners(image, patternSize=grid_size)
    assert(return_value)
    gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)
    corners = corners.reshape((88,2)).copy()
    criteria = (cv2.TERM_CRITERIA_MAX_ITER | cv2.TERM_CRITERIA_EPS, 100, 0.001)
    cv2.cornerSubPix(gray, corners, (5,5), (-1,-1), criteria)
    A = np.empty((0,9), dtype=float)
    O = np.array([0,0,0]).reshape(1,3)
    square_size = 11 # mm
    real_coords = []
    pix_coords = []
    for index, corner in enumerate(corners):
        Xpix = corner[0] # in pixel
        Ypix = corner[1]
        pix_coords.append(np.array([Xpix, Ypix]))
        grid_size_cv2 = tuple(reversed(grid_size)) # OpenCV and Python store matrices differently
        u_index, v_index = np.unravel_index(index, grid_size_cv2) # convert index from linear to 2D (0-based indexing)
        Xmm = u_index * square_size
        Ymm = v_index * square_size
        real_coords.append(np.array([Xmm, Ymm, 0, 1]))
        m = np.array([Xmm, Ymm, 1]).reshape(1,3)
        A = np.vstack((A, np.hstack((m,O,-Xpix*m))))
        A = np.vstack((A, np.hstack((O,m,-Ypix*m))))

    _, _, Vh = np.linalg.svd(A)
    h = Vh.transpose()[:,-1]
    H = h.reshape(3,3)
    return H, np.array(real_coords), np.array(pix_coords)

In [48]:
images_path = [os.path.join(folderpath, imagename) for imagename in os.listdir(folderpath) if imagename.endswith(".tiff")]
images_path.sort()

In [49]:
HH = []
for path in images_path:
    image = cv2.imread(path)
    H, _, _ = calculate_homography(image)
    HH.append(H)

2. per ogni omografia dalla relativa matrice costruire la matrice V + fare funzione che costruisce i vettori v, i, j

In [50]:
def vij_function (H, i, j):
  v=np.zeros(6)
  v[0]=H[0][i]*H[0][j]
  v[1]=H[0][i]*H[1][j]+H[1][i]*H[0][j]
  v[2]=H[1][i]*H[1][j]
  v[3]=H[2][i]*H[0][j]+H[0][i]*H[2][j]
  v[4]=H[2][i]*H[1][j]+H[1][i]*H[2][j]
  v[5]=H[2][i]*H[2][j]
  return v

In [51]:
VV=np.zeros((0,6))

for H in HH:
  v11=vij_function(H, 0, 0)
  v12=vij_function(H, 0, 1)
  v22=vij_function(H, 1, 1)
  V=np.array([v12.T, (v11-v22).T])
  print(V)
  VV=np.vstack((VV, V))
print(VV)

[[-2.13198825e-06  5.63903334e-06  2.09568388e-06  5.12549196e-10
  -9.45018978e-10 -2.77412891e-14]
 [-4.68694420e-06 -8.54923765e-06  6.69908224e-06  1.60720707e-09
   9.79675445e-10 -1.32207328e-13]]
[[-1.06526004e-05 -1.37588766e-05  9.07240259e-06 -4.38326809e-09
   7.63377199e-10 -2.11955853e-13]
 [ 1.20886053e-05 -3.95692999e-05 -1.52584212e-05 -2.36326391e-09
  -8.80077951e-09 -7.26739011e-13]]
[[-2.13944936e-06  2.17732965e-06  3.53923931e-06  1.65752629e-09
  -1.54964443e-09 -2.90579021e-13]
 [-4.41770795e-07 -1.14503640e-05  5.09573919e-06  1.36881802e-09
   4.05900856e-09 -4.70240903e-13]]
[[-1.02637565e-05 -1.21275206e-05  8.90787695e-06 -2.89834687e-09
  -2.98192578e-10 -1.64587134e-13]
 [ 1.09879517e-05 -3.83866705e-05 -1.31421909e-05  1.94842978e-10
  -5.62406050e-09 -1.48689635e-13]]
[[-1.24763542e-05 -3.32936609e-06  1.12964895e-05 -3.16967194e-09
   1.76450123e-10 -1.93520179e-13]
 [ 1.68655121e-06 -4.76110144e-05 -4.82553955e-06 -8.21952511e-10
  -6.13371064e-09 -1.

3. data V trovare B applicando la singular value decomposition

In [52]:
U, Sigma, S_T = np.linalg.svd(VV)
b = S_T.transpose()[:, -1]
B = np.empty((3, 3))
B[0, 0] = b[0]
B[0, 1] = B[1, 0] = b[1]
B[1, 1] = b[2]
B[0, 2] = B[2, 0] = b[3]
B[2, 1] = B[1, 2] = b[4]
B[2, 2] = b[5]
print(B)
eigvals, _ = np.linalg.eigh(B)
print(eigvals)

[[-2.68175698e-07 -4.03041606e-10  1.66921125e-04]
 [-4.03041606e-10 -2.68961779e-07  1.33910848e-04]
 [ 1.66921125e-04  1.33910848e-04 -9.99999977e-01]]
[-1.00000002e+00 -2.68265429e-07 -2.23077259e-07]


4. da B ottenere K e di conseguenza trovare R

In [53]:
B11 = b[0]
B12 = b[1]
B22 = b[2]
B13 = b[3]
B23 = b[4]
B33 = b[5]

v0 = (B12 * B13 - B11 * B23)/(B11 * B22 - B12 * B12)
l = B33 - (B13**2 + v0*(B12 * B13 - B11 * B13))/B11
alpha = np.sqrt(l/B11)
beta = np.sqrt(l * B11/(B11 * B22 - B12**2))
gamma = - B12 * (alpha**2) * beta / l
u0 = gamma * v0 / beta - B13 * (alpha**2) / l

K = np.array([[alpha, gamma, u0], [0, beta, v0], [0, 0, 1]])

K_inv = np.linalg.inv(K)
print(K_inv)
projections = []

for i, H in enumerate(HH):
    h1 = H[0, :]
    h2 = H[1, :]
    h3 = H[2, :]
    lambdas = 1/np.linalg.norm(K_inv@h1)
    r1 = lambdas * K_inv@h1
    r2 = lambdas * K_inv@h2
    t = lambdas * K_inv@h3
    t = t.reshape((-1, 1))
    r3 = np.cross(r1, r2)
    R = np.hstack((r1.reshape((-1, 1)), r2.reshape((-1, 1)), r3.reshape((-1, 1))))
    U, _ , V_t = np.linalg.svd(R)
    R = U@V_t
    P = K@np.hstack((R, t))
    print(P)
    



[[ 5.74236305e-04  8.63020493e-07 -3.57423030e-01]
 [ 0.00000000e+00  5.75076647e-04 -2.85783756e-01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.97249535e+02 -1.42729129e+03 -6.22460523e+02 -8.29003067e-08]
 [-2.58460244e+02  3.69911404e+02 -1.75131112e+03  4.11770031e-07]
 [ 8.79438609e-01  3.28946573e-01 -3.44066687e-01  9.96860539e-04]]
[[-6.07090942e+02  1.34450048e+03 -1.11484672e+03 -2.38830905e-07]
 [-5.00905216e+02  1.10934971e+03  1.33759633e+03  8.83451439e-07]
 [ 6.57518924e-01  7.53404655e-01 -7.09152214e-03  1.46514686e-03]]
[[-1.17248803e+03  1.42397275e+03 -1.29219139e+02  4.38626164e-07]
 [-2.43824287e+01  2.96150765e+01 -1.80810819e+03 -9.19141953e-07]
 [-9.04265856e-01 -3.32308760e-01 -2.68093546e-01 -1.06306995e-03]]
[[-6.50405725e+02  1.38681006e+03 -1.03578685e+03 -3.27390640e-07]
 [-4.86274561e+02  1.03685900e+03  1.39970970e+03  5.07125243e-07]
 [ 6.46683450e-01  7.62615246e-01  1.47817889e-02  1.49934912e-03]]
[[ 8.70269987e+02 -1.55436748e+03 -4.9

5. scegliamo un immagine, abbiamo bisogno di matrice di proiezione per l'immagine e i punti reali e proiettati dei corners della scacchiera che possiamo ricavare. Calcoliamo il reprojection error (pagina 45 lecture 3) (somma per ogni punto)

In [66]:
#reprojection error
def reprojection_error (P, R, I):
  epsilon=0
  epsilon_tot=0
  for i in range(R.shape[0]):
    u = np.dot(P[0], R[i])/np.dot(P[2], R[i])
    v = np.dot(P[1], R[i])/np.dot(P[2], R[i])

    print(I[i], u, v)
    epsilon = ((np.dot(P[0], R[i]) / np.dot(P[2], R[i]))-I[i][0])**2 + ((np.dot(P[1], R[i]) / np.dot(P[2], R[i]))-I[i][1])**2
    print(epsilon)
    epsilon_tot += epsilon
  return epsilon


image = cv2.imread(images_path[0])
_ , R, I = calculate_homography(image)
R = R/1000
I = I/1000
error = reprojection_error(P, R, I)
print(error)




[0.91219884 0.6373455 ] 0.001199660064826521 -0.0004065129177730856
1.2366471295542143
[0.87938464 0.6249572 ] -4421.621047859975 -6831.487431902601
66236269.772816926
[0.8469171  0.61263543] -4422.955404602673 -6833.549037409514
66275792.75199707
[0.81466866 0.60058796] -4423.400369171378 -6834.236515784576
66288676.93571158
[0.783023  0.5886812] -4423.622885031137 -6834.580306846724
66294902.734461606
[0.7514288 0.5767618] -4423.756405293069 -6834.7865980868955
66298461.83524048
[0.72019446 0.56496674] -4423.845423279006 -6834.924132498909
66300692.13145864
[0.6889171 0.553213 ] -4423.909009748115 -6835.0223747535365
66302160.430168614
[0.9241111 0.6047992] 598.4936893660454 924.6706130662342
1210987.0291716603
[0.89135504 0.59260106] -229.55081649096064 -354.67147304596693
179316.1567698259
[0.8587332 0.5804869] -823.2156364776148 -1271.8931729437109
2298287.781701853
[0.8264818 0.5685113] -1269.675719143257 -1961.681184920357
5464599.712995622
[0.79462427 0.5566728 ] -1617.64296369

6. superimposition di un oggetto per tutte le immagini